# Welcome to Lab: Simpson's Paradox 🔬

Before this lab section, you learned about Simpson's Paradox and confounding variables in lecture.  This week, you will find Simpson's Paradox through analysis of a dataset in Python, and get some practice writing conditionals for pandas DataFrames! 

A few tips to remember:

- Refer to your **lecture notebook** and the **pandas cheat sheet** to help you out with the code!
- If you find yourself stuck for more than a few minutes, ask a neighbor or course staff for help!  When you are giving help to your neighbor, explain the **idea and approach** to the problem without sharing the answer itself so they can have the same **<i>ah-hah</i>** moment!
- We are here to help you!  Don't feel embarrassed or shy to ask us for help!

Let's get started!

In [ ]:
# Meet your CAs and TA if you haven't already!
# ...first name is enough, we'll know who they are! :)
ta_name = "Daniel"
ca1_name = "Linh"
ca2_name = "Eeshan"


# Say hello to each other!
# - Groups of 3 are ideal :)
# - However, groups of 2 or 4 are fine too!
#
# QOTD to Ask Your Group: "Orange or Blue?"
partner1_name = "David Mocianko"
partner1_netid = "dmoci2"
partner1_orange_or_blue = "blue"

partner2_name = "Shane Moncayo"
partner2_netid = "moncayo3"
partner2_orange_or_blue = "blue"

partner3_name = "Evan Walker"
partner3_netid = "evanaw3"
partner3_orange_or_blue = "Orange"

<hr style="color: #DD3403;">

# Part 1: The GPA Dataset

Many of you have likely come across one of the GPA visualizations found at https://waf.cs.illinois.edu/discovery/gpa/, potentially out of 
curiosity or the need to investigate a mysterious GenEd class you signed up for. 

Regardless, these visualizations are built on the **GPA Dataset** of UIUC students across all course subjects. Today, you're going to do some analysis and discover a case of Simpson's Paradox within data taken from UIUC courses!

## Load the GPA Dataset

The most recent version of the "GPA Dataset" is available here:
```
https://raw.githubusercontent.com/wadefagen/datasets/master/gpa/uiuc-gpa-dataset.csv
```

Use Python to load this dataset into a DataFrame called `df`:

In [4]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/wadefagen/datasets/master/gpa/uiuc-gpa-dataset.csv")
df

Year    Term YearTerm Subject  Number                   Course Title  \
0      2021    Fall  2021-fa     AAS     100   Intro Asian American Studies   
1      2021    Fall  2021-fa     AAS     100   Intro Asian American Studies   
2      2021    Fall  2021-fa     AAS     100   Intro Asian American Studies   
3      2021    Fall  2021-fa     AAS     100   Intro Asian American Studies   
4      2021    Fall  2021-fa     AAS     100   Intro Asian American Studies   
...     ...     ...      ...     ...     ...                            ...   
61552  2010  Summer  2010-su    STAT     410  Statistics and Probability II   
61553  2010  Summer  2010-su    STAT     440    Statistical Data Management   
61554  2010  Summer  2010-su     TAM     212          Introductory Dynamics   
61555  2010  Summer  2010-su     TAM     251   Introductory Solid Mechanics   
61556  2010  Summer  2010-su     TAM     335   Introductory Fluid Mechanics   

      Sched Type  A+   A  A-  ...  B-  C+  C  C-  D+  D  D-  F  W  \
0            DIS   2  14   2  ...   3   0  1   0   0  0   0  0  0   
1            DIS   0  15   0  ...   1   0  2   0   0  0   0  1  0   
2             OD   7   4   1  ...   0   2  3   0   0  1   0  1  0   
3            DIS   1  18   0  ...   1   0  0   0   0  0   0  0  0   
4            DIS   0  16   1  ...   1   0  2   0   0  0   0  0  0   
...          ...  ..  ..  ..  ...  ..  .. ..  ..  .. ..  .. .. ..   
61552        LEC   5  10   2  ...   1   0  1   3   0  0   0  2  1   
61553        LEC   4  12   8  ...   0   0  0   0   0  0   0  0  0   
61554        LEC   0   1   3  ...   7   5  1   1   0  2   0  1  0   
61555        LCD   1   2   2  ...   0   3  3   2   0  0   1  1  0   
61556        LEC   5   8   9  ...   1   0  4   0   0  1   0  2  0   

             Primary Instructor  
0                 Zheng, Reanne  
1       Atienza, Paul Michael L  
2                      Wang, Yu  
3                 Zheng, Reanne  
4       Atienza, Paul Michael L  
...                         ...  
61552        Stepanov, Alexei G  
61553              Unger, David  
61554         Morgan, William T  
61555  Ott-Monsivais, Stephanie  
61556          Keane, Richard D  

[61557 rows x 22 columns]

### 🔬 Test Case Checkpoint 🔬

In [2]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert(len(df) == 61557 ), "This is not the GPA dataset you're looking for"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


## Data Cleaning

The GPA dataset contains only raw GPA data and is not the easiest to work with if we want to analyze intricacies such as the average GPA or passing rate.  Luckily, DataFrames are modifiable, so we can add more columns based on what questions we want to answer.

The process of modifying a dataset via deletion (cleaning up empty/unwanted values) or addition (adding new columns) is often called **data cleaning**. This is an important concept in Data Science, because you won't always receive your data in the perfect format for your purposes.

With the GPA dataset, we want to investigate one innocent question posed by a theoretical incoming student: 
- *Is it easier to get an A in STAT or CS courses at UIUC?*

To do this, we'll need to first perform some modifications on our loaded DataFrame, `df`. Specifically, we will need to create two new columns:
- A `Total` column,  the total number of students in every course
- An `A_Grade` column, the total number of students receiving an A+, A, or A- in every course

### Puzzle 1.1: Creating a `Total Students` Column

Create the new column `Total Students` that stores the total number of students in every course.

- The `Total Students` column should include every grade **except** `W`.  W means the student withdrew from the course. We don't want to include withdraws in our analysis.

In [10]:
df['Total Students'] = (df["A+"]+df["A"]+df["A-"]+df["B+"]+df["B"]+df["B-"]+df["C+"]+df["C"]+df["C-"]+df["D+"]+df["D"]+df["D-"]+df["F"])
df['Total Students']

0        30
1        27
2        27
3        26
4        28
         ..
61552    31
61553    28
61554    28
61555    21
61556    39
Name: Total Students, Length: 61557, dtype: int64

**Note:** Our DataFrame is so large that you may have to scroll to the right to see the new column, `Total Students`.

### Puzzle 1.2: Creating an `A_Grade` Column

Create the new column `A_Grade` that stores the total number of "A"s given in every course.

- We consider an "A" to be any type of A, in other words "A+", "A", or an "A-" are all included.

In [11]:
df['A_Grade'] = (df["A+"]+df["A"]+df["A-"])
df['A_Grade']

0        18
1        15
2        12
3        19
4        17
         ..
61552    17
61553    24
61554     4
61555     5
61556    22
Name: A_Grade, Length: 61557, dtype: int64

### 🔬 Test Case Checkpoint 🔬

In [12]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert('A_Grade' in df), "Make sure you've named the A_Grade column properly and added it to the dataframe"
assert(df['A_Grade'].sum() == 1983288), "Double check the values of your A_Grade column, and make sure you are calling .sum() on the correct list (A_Grade)"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


# Part 2: Working with the GPA Dataset
Now that we have our columns set up, we can perform basic mathematical analysis on the DataFrame to get insight towards our question. 

Should be simple, right? Let's just see which subject, STAT or CS, has a greater percentage of A grades .

### Puzzle 2.1: Subject DataFrames

Select only the rows of the GPA dataset `df` with a `Subject` of `STAT`. Assign these rows to a new DataFrame, `df_STAT`. 
- Make sure your result only contains STAT courses!

In [15]:
df_STAT = df[df.Subject=="STAT"]
df_STAT

Year    Term YearTerm Subject  Number                   Course Title  \
2576   2021    Fall  2021-fa    STAT     100                     Statistics   
2577   2021    Fall  2021-fa    STAT     100                     Statistics   
2578   2021    Fall  2021-fa    STAT     107         Data Science Discovery   
2579   2021    Fall  2021-fa    STAT     200           Statistical Analysis   
2580   2021    Fall  2021-fa    STAT     207       Data Science Exploration   
...     ...     ...      ...     ...     ...                            ...   
61549  2010  Summer  2010-su    STAT     100                     Statistics   
61550  2010  Summer  2010-su    STAT     100                     Statistics   
61551  2010  Summer  2010-su    STAT     400   Statistics and Probability I   
61552  2010  Summer  2010-su    STAT     410  Statistics and Probability II   
61553  2010  Summer  2010-su    STAT     440    Statistical Data Management   

      Sched Type   A+    A  A-  ...   C  C-  D+   D  D-   F  W  \
2576         LCD  150   78  49  ...  18  10   5   5   3  18  3   
2577         ONL  208  151  81  ...  42  24  17  25   5  45  9   
2578         OLC  127   61  33  ...   5   4   2   2   0  12  2   
2579         ONL   75  135  34  ...   9   3   0   0   1   3  0   
2580         LEC   17    3   2  ...   1   0   0   0   1   3  1   
...          ...  ...  ...  ..  ...  ..  ..  ..  ..  ..  .. ..   
61549        LCD    1   13   3  ...   2   2   3   1   1   1  0   
61550        LCD    0   14   3  ...   3   0   0   1   0   3  1   
61551        LEC    4   15   7  ...   2   2   0   1   0   3  1   
61552        LEC    5   10   2  ...   1   3   0   0   0   2  1   
61553        LEC    4   12   8  ...   0   0   0   0   0   0  0   

        Primary Instructor  Total Students  A_Grade  
2576     Flanagan, Karle A             442      277  
2577     Flanagan, Karle A             836      440  
2578     Flanagan, Karle A             287      221  
2579      Fireman, Ellen S             322      244  
2580   Ellison, Victoria M              29       22  
...                    ...             ...      ...  
61549   Hirtz, Nathaniel R              39       17  
61550     Dalpiaz, David M              28       17  
61551       Monrad, Ditlev              47       26  
61552   Stepanov, Alexei G              31       17  
61553         Unger, David              28       24  

[884 rows x 24 columns]

Select only the rows of the GPA dataset `df` with a `Subject` of `CS`. Assign these rows to a new dataframe, `df_CS`. 
- Make sure your result only contains CS courses!

In [16]:
df_CS = df[df.Subject=="CS"]
df_CS

Year    Term YearTerm Subject  Number                   Course Title  \
800    2021    Fall  2021-fa      CS     100           Freshman Orientation   
801    2021    Fall  2021-fa      CS     100           Freshman Orientation   
802    2021    Fall  2021-fa      CS     101   Intro Computing: Engrg & Sci   
803    2021    Fall  2021-fa      CS     105      Intro Computing: Non-Tech   
804    2021    Fall  2021-fa      CS     105      Intro Computing: Non-Tech   
...     ...     ...      ...     ...     ...                            ...   
61433  2010  Summer  2010-su      CS     101   Intro Computing: Engrg & Sci   
61434  2010  Summer  2010-su      CS     225                Data Structures   
61435  2010  Summer  2010-su      CS     373          Theory of Computation   
61436  2010  Summer  2010-su      CS     421  Progrmg Languages & Compilers   
61437  2010  Summer  2010-su      CS     598  Object-Oriented Prog & Design   

      Sched Type   A+    A  A-  ...   C  C-  D+  D  D-   F  W  \
800          LEC    0  246   7  ...   0   0   0  0   0   2  0   
801          OLC    0  223   6  ...   1   0   0  2   0   6  2   
802          OLC  112  264  48  ...   7   5   4  2   2  10  2   
803          OLC   27   83  53  ...  20  11   9  6   1  24  9   
804          OLC   23  100  37  ...  19   9   6  5   4  14  6   
...          ...  ...  ...  ..  ...  ..  ..  .. ..  ..  .. ..   
61433        LBD    4    6   2  ...   0   1   0  0   0   0  0   
61434        LBD    1    5   1  ...   3   1   0  2   0   1  0   
61435        LEC    5    1   5  ...   0   2   0  2   0   1  1   
61436        LCD    2    5   5  ...   1   4   0  4   0   0  1   
61437        ONL    3   22   0  ...   0   0   0  0   0   0  0   

       Primary Instructor  Total Students  A_Grade  
800          Gunter, Elsa             261      253  
801          Gunter, Elsa             241      229  
802         Davis, Neal E             520      424  
803         Zilles, Craig             348      163  
804         Zilles, Craig             342      160  
...                   ...             ...      ...  
61433   Gambill, Thomas N              21       12  
61434       Earls, John C              21        7  
61435        Kumar, Viraj              26       11  
61436      Hafiz, Munawar              30       12  
61437    Johnson, Ralph E              30       25  

[1977 rows x 24 columns]

### 🔬 Test Case Checkpoint 🔬

In [17]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any error our output, you PASSED all test cases!
# - If this cell results in any errors, check you previous cell, make changes, and RE-RUN your code and then this cell.
assert( 'df_STAT' in vars() ), "Make certain to name the STAT courses df_STAT."
assert( 'df_CS' in vars() ), "Make certain to name the CS courses df_CS."
assert( len(df_STAT[df_STAT.Subject != "STAT"] ) == 0 ), "It looks like you did not subset df_STAT to only STAT courses."
assert( len(df_CS[df_CS.Subject != "CS"] ) == 0 ), "It looks like you did not subset df_STAT to only STAT courses."

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Puzzle 2.2: Comparing Overall Percentages

With our two new dataframes of STAT and CS courses, use the following code cell to determine the **percentage** of A grades recieved in STAT and CS courses, storing them as variables `stat_a` and `cs_a` respectively. 

Print statements have been provided to show the values you calculate. 

**Hint:** To find the % of As, divide the total number of students by the total number of As. Remember the function `sum(df['column_name'])` can be used to add up the values of all rows in a particular column of a dataframe.

In [28]:
stat_a =  sum(df_STAT["A_Grade"]) / sum(df_STAT["Total Students"])
print(f'Overall % of As in STAT is: {stat_a}')

cs_a = sum(df_CS["A_Grade"]) / sum(df_CS["Total Students"])
print(f'Overall % of As in CS is: {cs_a}')

Overall % of As in STAT is: 0.6027997446909787
Overall % of As in CS is: 0.5891362965182221


### 🔬 Test Case Checkpoint 🔬

In [29]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
import math
assert(math.isclose(stat_a, 0.6027997446909787)), "The overall percentage of A grades recieved in STAT courses does not appear to have been correctly calculated"
assert(math.isclose(cs_a, 0.5891362965182221)), "The overall percentage of A grades recieved in CS courses does not appear to have been correctly calculated"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Analysis: Good Comparison?
**Q: What conclusion can you take from the overall percentages found above when asking the original question:**
- *Is it easier to get an A in STAT or CS courses at UIUC?*

*(✏️ It is easier to get an A in STAT courses. ✏️)*

**Q: Given what you've learned about experimental design, what are some reasons (specific to this dataset) you may not trust this conclusion? If you would trust it, explain why.**

*(✏️ I do not trust this conclusion. I feel that it varies per year and I feel it varies per course. I feel that individual STAT/CS Courses skew the data. I think that one STAT course might be a big class that gives out easy As and that skews the data, showing that STAT courses get easier As even though that may not necessarily be the case for the whole major. ✏️)*

## An Extra Consideration

If you look at the `Year` column of our GPA dataset, you might notice that we have some old data in our set - all the way back to **2010**! This means we aren't really answering our question from the perspective of a student now. 

If we want to know if it is easier to **currently** get an A in a STAT or CS course, we should control for the **date** of the data by looking at more **recent years** specifically.

### Puzzle 2.3: More DataFrames

Using the code cells below, define four new DataFrames by selecting from rows of our previously created `df_CS` and `df_STAT`. 

- `df_cs_recent`: all `CS` course data in *recent years*
- `df_stat_recent`: all `STAT` course data in *recent years*
- `df_cs_old`: all other, older `CS` course data
- `df_stat_old`: all other, older `STAT` course data

Define *recent years* as **any year after and including 2020**, and older years as any year before 2020. 

Feel free to use row selection conditionals OR the `.isin()` function you learned in the last lab.

In [32]:
df_stat_recent = df[(df.Subject == "STAT")&(df.Year >= 2020)]
df_stat_recent

Year    Term YearTerm Subject  Number                   Course Title  \
2576  2021    Fall  2021-fa    STAT     100                     Statistics   
2577  2021    Fall  2021-fa    STAT     100                     Statistics   
2578  2021    Fall  2021-fa    STAT     107         Data Science Discovery   
2579  2021    Fall  2021-fa    STAT     200           Statistical Analysis   
2580  2021    Fall  2021-fa    STAT     207       Data Science Exploration   
...    ...     ...      ...     ...     ...                            ...   
9936  2020  Winter  2020-wi    STAT     420      Statistical Modeling in R   
9937  2020  Winter  2020-wi    STAT     420  Methods of Applied Statistics   
9938  2020  Winter  2020-wi    STAT     420  Methods of Applied Statistics   
9939  2020  Winter  2020-wi    STAT     440    Statistical Data Management   
9940  2020  Winter  2020-wi    STAT     448         Advanced Data Analysis   

     Sched Type   A+    A  A-  ...   C  C-  D+   D  D-   F  W  \
2576        LCD  150   78  49  ...  18  10   5   5   3  18  3   
2577        ONL  208  151  81  ...  42  24  17  25   5  45  9   
2578        OLC  127   61  33  ...   5   4   2   2   0  12  2   
2579        ONL   75  135  34  ...   9   3   0   0   1   3  0   
2580        LEC   17    3   2  ...   1   0   0   0   1   3  1   
...         ...  ...  ...  ..  ...  ..  ..  ..  ..  ..  .. ..   
9936        ONL   28  123  18  ...   1   0   0   0   0   1  0   
9937        ONL    5   14   5  ...   0   0   0   0   0   1  0   
9938        ONL   10   48  11  ...   1   0   0   0   0   2  1   
9939        ONL    0   29   0  ...   0   0   0   0   0   3  1   
9940        ONL    5    8   0  ...   1   0   0   0   0   1  0   

          Primary Instructor  Total Students  A_Grade  
2576       Flanagan, Karle A             442      277  
2577       Flanagan, Karle A             836      440  
2578       Flanagan, Karle A             287      221  
2579        Fireman, Ellen S             322      244  
2580     Ellison, Victoria M              29       22  
...                      ...             ...      ...  
9936            Unger, David             185      169  
9937            Unger, David              25       24  
9938            Unger, David              86       69  
9939   Kinson, Christopher L              36       29  
9940  Bravo De Guenni, Lelys              21       13  

[186 rows x 24 columns]

In [33]:
df_cs_recent = df[(df.Subject == "CS") & (df.Year >= 2020)]
df_cs_recent

Year    Term YearTerm Subject  Number                  Course Title  \
800   2021    Fall  2021-fa      CS     100          Freshman Orientation   
801   2021    Fall  2021-fa      CS     100          Freshman Orientation   
802   2021    Fall  2021-fa      CS     101  Intro Computing: Engrg & Sci   
803   2021    Fall  2021-fa      CS     105     Intro Computing: Non-Tech   
804   2021    Fall  2021-fa      CS     105     Intro Computing: Non-Tech   
...    ...     ...      ...     ...     ...                           ...   
9785  2020  Winter  2020-wi      CS     498             Data Visualiztion   
9786  2020  Winter  2020-wi      CS     498             Data Visualiztion   
9787  2020  Winter  2020-wi      CS     513  Theory & Pract Data Cleaning   
9788  2020  Winter  2020-wi      CS     598                Cloud Capstone   
9789  2020  Winter  2020-wi      CS     598          Data Mining Capstone   

     Sched Type   A+    A  A-  ...   C  C-  D+  D  D-   F  W  \
800         LEC    0  246   7  ...   0   0   0  0   0   2  0   
801         OLC    0  223   6  ...   1   0   0  2   0   6  2   
802         OLC  112  264  48  ...   7   5   4  2   2  10  2   
803         OLC   27   83  53  ...  20  11   9  6   1  24  9   
804         OLC   23  100  37  ...  19   9   6  5   4  14  6   
...         ...  ...  ...  ..  ...  ..  ..  .. ..  ..  .. ..   
9785        ONL  226  169  30  ...   3   0   3  0   0   9  0   
9786        OLC   20   14   3  ...   0   0   0  0   0   1  2   
9787        ONL  350   28   3  ...   0   0   0  0   0   1  0   
9788        ONL   35    6   0  ...   0   0   0  0   0   1  0   
9789        ONL   28   19   2  ...   1   0   0  0   0   0  0   

       Primary Instructor  Total Students  A_Grade  
800          Gunter, Elsa             261      253  
801          Gunter, Elsa             241      229  
802         Davis, Neal E             520      424  
803         Zilles, Craig             348      163  
804         Zilles, Craig             342      160  
...                   ...             ...      ...  
9785         Hart, John C             476      425  
9786         Hart, John C              46       37  
9787  Ludaescher, Bertram             390      381  
9788        Farivar, Reza              43       41  
9789        Farivar, Reza              57       49  

[319 rows x 24 columns]

In [34]:
df_stat_old = df[(df.Subject == "STAT") & (df.Year < 2020)] 
df_stat_old

Year    Term YearTerm Subject  Number                   Course Title  \
12452  2019    Fall  2019-fa    STAT     100                     Statistics   
12453  2019    Fall  2019-fa    STAT     100                     Statistics   
12454  2019    Fall  2019-fa    STAT     100                     Statistics   
12455  2019    Fall  2019-fa    STAT     107         Data Science Discovery   
12456  2019    Fall  2019-fa    STAT     200           Statistical Analysis   
...     ...     ...      ...     ...     ...                            ...   
61549  2010  Summer  2010-su    STAT     100                     Statistics   
61550  2010  Summer  2010-su    STAT     100                     Statistics   
61551  2010  Summer  2010-su    STAT     400   Statistics and Probability I   
61552  2010  Summer  2010-su    STAT     410  Statistics and Probability II   
61553  2010  Summer  2010-su    STAT     440    Statistical Data Management   

      Sched Type  A+    A  A-  ...   C  C-  D+   D  D-   F  W  \
12452        ONL  69  105  63  ...  25  12   4  10   1  13  2   
12453        LCD  71  143  95  ...  24  19  19  14   5   8  4   
12454        ONL  19   72  23  ...  19   3   5   4   3   9  1   
12455        LEC   6  109   7  ...   3   3   4   2   0   5  2   
12456        LCD  28    1   1  ...   0   1   0   0   0   1  0   
...          ...  ..  ...  ..  ...  ..  ..  ..  ..  ..  .. ..   
61549        LCD   1   13   3  ...   2   2   3   1   1   1  0   
61550        LCD   0   14   3  ...   3   0   0   1   0   3  1   
61551        LEC   4   15   7  ...   2   2   0   1   0   3  1   
61552        LEC   5   10   2  ...   1   3   0   0   0   2  1   
61553        LEC   4   12   8  ...   0   0   0   0   0   0  0   

       Primary Instructor  Total Students  A_Grade  
12452   Flanagan, Karle A             477      237  
12453   Flanagan, Karle A             589      309  
12454          Yu, Albert             254      114  
12455   Flanagan, Karle A             170      122  
12456  Simpson, Douglas G              39       30  
...                   ...             ...      ...  
61549  Hirtz, Nathaniel R              39       17  
61550    Dalpiaz, David M              28       17  
61551      Monrad, Ditlev              47       26  
61552  Stepanov, Alexei G              31       17  
61553        Unger, David              28       24  

[698 rows x 24 columns]

In [35]:
df_cs_old = df[(df.Subject == "CS") & (df.Year < 2020)] 
df_cs_old

Year    Term YearTerm Subject  Number                   Course Title  \
10833  2019    Fall  2019-fa      CS     100           Freshman Orientation   
10834  2019    Fall  2019-fa      CS     101   Intro Computing: Engrg & Sci   
10835  2019    Fall  2019-fa      CS     105      Intro Computing: Non-Tech   
10836  2019    Fall  2019-fa      CS     125      Intro to Computer Science   
10837  2019    Fall  2019-fa      CS     126         Software Design Studio   
...     ...     ...      ...     ...     ...                            ...   
61433  2010  Summer  2010-su      CS     101   Intro Computing: Engrg & Sci   
61434  2010  Summer  2010-su      CS     225                Data Structures   
61435  2010  Summer  2010-su      CS     373          Theory of Computation   
61436  2010  Summer  2010-su      CS     421  Progrmg Languages & Compilers   
61437  2010  Summer  2010-su      CS     598  Object-Oriented Prog & Design   

      Sched Type   A+    A  A-  ...   C  C-  D+   D  D-   F  W  \
10833        LEC    0  214  22  ...  12  10   4   2   2   4  0   
10834        LEC  282  135  62  ...   6   3   4   3   5  12  0   
10835        LEC   86  134  87  ...  37  14  12  10  10  14  4   
10836        LEC  150  357  45  ...  11   6   0  13   0  23  3   
10837        LCD    9   56  37  ...   3   1   3   0   2   3  2   
...          ...  ...  ...  ..  ...  ..  ..  ..  ..  ..  .. ..   
61433        LBD    4    6   2  ...   0   1   0   0   0   0  0   
61434        LBD    1    5   1  ...   3   1   0   2   0   1  0   
61435        LEC    5    1   5  ...   0   2   0   2   0   1  1   
61436        LCD    2    5   5  ...   1   4   0   4   0   0  1   
61437        ONL    3   22   0  ...   0   0   0   0   0   0  0   

        Primary Instructor  Total Students  A_Grade  
10833         Gunter, Elsa             341      236  
10834        Davis, Neal E             610      479  
10835     Harris, Albert F             601      307  
10836  Challen, Geoffrey W             671      552  
10837   Woodley, Michael J             145      102  
...                    ...             ...      ...  
61433    Gambill, Thomas N              21       12  
61434        Earls, John C              21        7  
61435         Kumar, Viraj              26       11  
61436       Hafiz, Munawar              30       12  
61437     Johnson, Ralph E              30       25  

[1658 rows x 24 columns]

### 🔬 Test Case Checkpoint 🔬

In [36]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any error our output, you PASSED all test cases!
# - If this cell results in any errors, check you previous cell, make changes, and RE-RUN your code and then this cell.
assert( 'df_stat_recent' in vars() ), "Make certain to name the recent STAT courses df_stat_recent."
assert( 'df_cs_recent' in vars() ), "Make certain to name the recent CS courses df_cs_recent."
assert( 'df_stat_old' in vars() ), "Make certain to name the old STAT courses df_stat_old."
assert( 'df_cs_old' in vars() ), "Make certain to name the old CS courses df_cs_old."

assert( len(df_stat_recent[df_stat_recent.Year < 2020] ) == 0 ), "Make sure only years after and including 2020 are in the df_stat_recent dataframe."
assert( len(df_cs_recent[df_cs_recent.Year < 2020] ) == 0 ), "Make sure only years after and including 2020 are in the df_cs_recent dataframe."
assert( len(df_stat_old[df_stat_old.Year >= 2020] ) == 0 ), "Make sure only years before 2020 are in the df_stat_old dataframe."
assert( len(df_cs_old[df_cs_old.Year >= 2020] ) == 0 ), "Make sure only years before 2020 are in the df_cs_old dataframe."

assert(len(df[ df.index.isin(df_stat_recent.index) & df.index.isin(df_stat_old.index) ]) == 0), "Check for duplicate values in your df_stat_recent and df_stat_old dataframes."
assert(len(df[ df.index.isin(df_cs_recent.index) & df.index.isin(df_cs_old.index) ]) == 0), "Check for duplicate values in your df_cs_recent and df_cs_old dataframes."

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Puzzle 2.4: New Analysis
Now that we've got all the DataFrames setup with GPA data of the CS and STAT courses separated by recency (2020 or newer being 'recent'), we can do more in-depth analysis to investigate our question. 

In the following code cells, **calculate the percentages** described by the comment. Your answer should be a decimal between 0 and 1. 

**Hint:** the function `sum(df['column_name'])` can be used to add up the values of all rows in a particular column of a DataFrame.

In [38]:
# Percentage of As received in CS in recent years
cs_recent_a = sum(df_cs_recent["A_Grade"]) / sum(df_cs_recent["Total Students"])
print(f'Percentage of As received in CS in recent years: {cs_recent_a}')

Percentage of As received in CS in recent years: 0.7305164527732138


In [39]:
# Percentage of As received in STAT in recent years
stat_recent_a = sum(df_stat_recent["A_Grade"]) / sum(df_stat_recent["Total Students"])
print(f'Percentage of As received in STAT in recent years: {stat_recent_a}')

Percentage of As received in STAT in recent years: 0.683119837776349


In [40]:
# percentage of As recieved in CS in older years
cs_old_a = sum(df_cs_old["A_Grade"]) / sum(df_cs_old["Total Students"])
print(f'Percentage of As received in CS in older years: {cs_old_a}') 

Percentage of As received in CS in older years: 0.5442563329057359


In [41]:
# percentage of As recieved in CS in older years
stat_old_a = sum(df_stat_old["A_Grade"]) / sum(df_stat_old["Total Students"])
print(f'Percentage of As received in STAT in older years: {stat_old_a}')

Percentage of As received in STAT in older years: 0.5803123874934227


### 🔬 Test Case Checkpoint 🔬

In [42]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert(math.isclose(cs_recent_a,  0.7305164527732138)), "The overall percentage of A grades recieved in STAT courses recently does not appear to have been correctly calculated"
assert(math.isclose(stat_recent_a, 0.683119837776349)), "The overall percentage of A grades recieved in CS courses recently does not appear to have been correctly calculated"

assert(math.isclose(cs_old_a,  0.5442563329057359)), "The overall percentage of A grades recieved in STAT courses in older years does not appear to have been correctly calculated"
assert(math.isclose(stat_old_a, 0.5803123874934227)), "The overall percentage of A grades recieved in CS courses in older years does not appear to have been correctly calculated"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Observe the Results
Run the following cell to format all of your answers as a DataFrame, keeping in mind that "Old" means data from courses before 2020, and "New" means courses held during or after 2020. 

In [43]:
pd.DataFrame([
  {'Old % of A': cs_old_a, 'New % of A': cs_recent_a, 'Overall % of A': cs_a},
  {'Old % of A': stat_old_a, 'New % of A': stat_recent_a, 'Overall % of A': stat_a}
], index=['CS', 'STAT'])

Old % of A  New % of A  Overall % of A
CS      0.544256    0.730516        0.589136
STAT    0.580312    0.683120        0.602800

Notice that when observing the overall % of A grades received, you may think `STAT` is easier than `CS` to get an A in. But in the sub-group of the courses held in years of 2020 and later, we see that CS actually has a higher A-Grade rate! 

This is **Simpson's Paradox**: a pattern within a population can appear, disappear, or reverse when you look at subpopulations.

In more formal terms, Simpson's Paradox can cause you to observe a pattern reverse when you look at the overall group statistics versus statistics of groups post-stratification. In this case we are stratifying by time.


### Analysis: Reflecting on New Observations 

You should see the pattern reverse when you look at the overall A grade percentages vs. the percentages stratified to account for recency. This is called **Simpson's Paradox**: a pattern within a population can appear, disappear, or reverse when you look at subpopulations.

Now think about how would you now respond differently to the incoming student's question:
- *Is it easier to get an A in STAT or CS courses at UIUC?*

**Q: Which comparison of percentages do you trust more and why? Are there any other potential confounding variables when answering this question that could be investigated further? Respond with at least three full sentences.**

*(✏️ I trust the more recent comparison of percentages more. I believe they are more relevant to students now and incoming student's are more likely to fit this trend more. There are many factors that could have affected the previous comparisons but the factors that affect the recent comparisons are more relatable and relevant to incoming students. ✏️)*

<hr style="color: #DD3403;">

# Part 3: Revisiting the Hello Dataset

Over the past two weeks, you created a series of questions that made up the "Hello Dataset" and completed the survey by answering all of the questions yourself. Now, we will load this dataset again and briefly answer a few questions.

## Load the Hello Dataset

The "Hello Dataset" is available here:
```
https://waf.cs.illinois.edu/discovery/hello-fa22.csv
```

Use Python to load this dataset into a DataFrame called `df_hello`:

In [44]:
import pandas as pd

df_hello = pd.read_csv("https://waf.cs.illinois.edu/discovery/hello-fa22.csv")
df_hello

Name  Lab  Number 1 or Number 2?  Number 3 or Number 5?  \
0          CA Ana  AL1                    2.0                    5.0   
1          CA Ram  AYR                    2.0                    5.0   
2          CA Xin  AYO                    1.0                    5.0   
3           Lekha  AYH                    2.0                    2.0   
4            Reni  AYD                    2.0                    5.0   
..            ...  ...                    ...                    ...   
519          Bill  AYC                    1.0                    5.0   
520       Renault  AYR                    2.0                    5.0   
521  Shivam Patel  AYC                    1.0                    3.0   
522         Colin  AYI                    1.0                    5.0   
523          Evan  AYG                    2.0                    3.0   

    What is your preferred way of saying one half? 7UP or Sierra Mist?  \
0                                            2-Jan         Sierra Mist   
1                                            2-Jan                 7UP   
2                                            2-Jan         Sierra Mist   
3                                              0.5                 7UP   
4                                            2-Jan                 7UP   
..                                             ...                 ...   
519                                          2-Jan         Sierra Mist   
520                                            0.5         Sierra Mist   
521                                            0.5                 7UP   
522                                          2-Jan         Sierra Mist   
523                                          2-Jan                 7UP   

    Android or iOS? Apple or Orange? Beef or Pork? Your birthday is...  ...  \
0               iOS            Apple          Pork   On/Before June 30  ...   
1               iOS            Apple           NaN     On/After July 1  ...   
2               iOS            Apple          Beef   On/Before June 30  ...   
3               iOS           Orange          Pork     On/After July 1  ...   
4               iOS            Apple          Pork   On/Before June 30  ...   
..              ...              ...           ...                 ...  ...   
519             iOS            Apple          Beef     On/After July 1  ...   
520         Android           Orange          Pork   On/Before June 30  ...   
521             iOS            Apple           NaN   On/Before June 30  ...   
522             iOS           Orange          Beef   On/Before June 30  ...   
523         Android           Orange          Beef     On/After July 1  ...   

    What is your favorite coffee shop drink?  \
0                                  Cold brew   
1                               Black coffee   
2                                  starbucks   
3                        pumpkin spice latte   
4                                      Mocha   
..                                       ...   
519                                      NaN   
520                                     moca   
521                          Don’t have one    
522                                      tea   
523                                      Tea   

    What is your favorite mobile app?  What is your favorite subject?  \
0                            Snapchat                            Math   
1                      Google chrome                      Mathematics   
2                           Starbucks                      Psychology   
3                             twitter               english (writing)   
4                           Instagram                            Math   
..                                ...                             ...   
519                               NaN                             NaN   
520                           wechat                              cs    
521                           Youtube                  World History    
522

### 🔬 Test Case Checkpoint 🔬

In [45]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert(len(df_hello) == 524), "This is not the Hello dataset you're looking for. Check the URL."

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


## Classes v. Sleep

With the Hello Dataset, we are going to briefly explore two quantitative questions:
- How many hours of sleep do you get on average? 
- How many classes are you taking this semester?

### Puzzle 3.1: Observation Subsets

In this situation, let's define an average of **6 or more hours** of sleep as "good sleep" and any less to be defined as "bad sleep".

From this, create two DataFrames that contain subsets of the Hello Dataset: 
- `df_goodsleep`: including everyone who gets "good sleep" on average
- `df_badsleep`: including everyone who gets "bad sleep" on average

In [49]:
sleep = 'How many hours of sleep do you get on average?'
df_goodsleep = df_hello[(df_hello["How many hours of sleep do you get on average?"] >= 6)]
df_goodsleep

Name  Lab  Number 1 or Number 2?  Number 3 or Number 5?  \
0     CA Ana  AL1                    2.0                    5.0   
2     CA Xin  AYO                    1.0                    5.0   
3      Lekha  AYH                    2.0                    2.0   
4       Reni  AYD                    2.0                    5.0   
5     Linger  AYF                    2.0                    5.0   
..       ...  ...                    ...                    ...   
517   Lauren  AYP                    1.0                    3.0   
518  Celesta  AYS                    1.0                    3.0   
519     Bill  AYC                    1.0                    5.0   
520  Renault  AYR                    2.0                    5.0   
522    Colin  AYI                    1.0                    5.0   

    What is your preferred way of saying one half? 7UP or Sierra Mist?  \
0                                            2-Jan         Sierra Mist   
2                                            2-Jan         Sierra Mist   
3                                              0.5                 7UP   
4                                            2-Jan                 7UP   
5                                            2-Jan                 7UP   
..                                             ...                 ...   
517                                          2-Jan                 7UP   
518                                            0.5                 7UP   
519                                          2-Jan         Sierra Mist   
520                                            0.5         Sierra Mist   
522                                          2-Jan         Sierra Mist   

    Android or iOS? Apple or Orange? Beef or Pork? Your birthday is...  ...  \
0               iOS            Apple          Pork   On/Before June 30  ...   
2               iOS            Apple          Beef   On/Before June 30  ...   
3               iOS           Orange          Pork     On/After July 1  ...   
4               iOS            Apple          Pork   On/Before June 30  ...   
5               iOS            Apple          Beef   On/Before June 30  ...   
..              ...              ...           ...                 ...  ...   
517             iOS            Apple          Beef   On/Before June 30  ...   
518             iOS            Apple          Beef     On/After July 1  ...   
519             iOS            Apple          Beef     On/After July 1  ...   
520         Android           Orange          Pork   On/Before June 30  ...   
522             iOS           Orange          Beef   On/Before June 30  ...   

    What is your favorite coffee shop drink?  \
0                                  Cold brew   
2                                  starbucks   
3                        pumpkin spice latte   
4                                      Mocha   
5                                 Starbucks    
..                                       ...   
517                                      tea   
518                              Frappuccino   
519                                      NaN   
520                                     moca   
522                                      tea   

    What is your favorite mobile app? What is your favorite subject?  \
0                            Snapchat                           Math   
2                           Starbucks                     Psychology   
3                             twitter              english (writing)   
4                           Instagram                           Math   
5                          App Store                       Any STEM    
..                                ...                            ...   
517                         instagram                        english   
518                         instagram                        science   
519                               NaN                            NaN   
520                           wechat                             cs 

In [50]:
df_badsleep = df_hello[(df_hello["How many hours of sleep do you get on average?"] < 6)]
df_badsleep

Name  Lab  Number 1 or Number 2?  Number 3 or Number 5?  \
1          CA Ram  AYR                    2.0                    5.0   
8         Suvinay  AYP                    1.0                    5.0   
16        Claudia  AL1                    2.0                    3.0   
17           Wing  AYB                    1.0                    5.0   
20       Sakshyam  AYT                    1.0                    5.0   
85         Rachel  AYK                    1.0                    5.0   
90           Lily  AYB                    1.0                    5.0   
156          rafy  AYK                    1.0                    3.0   
179        Alisha  AYC                    1.0                    5.0   
247         Oddie  AYR                    1.0                    3.0   
249         Sneha  AYD                    2.0                    5.0   
284          alex  AYO                    2.0                    5.0   
286         JACKY  AYA                    1.0                    3.0   
315         Rohan  AYQ                    2.0                    3.0   
316            Jo  AYM                    1.0                    3.0   
354         Vikas  AYE                    1.0                    3.0   
441         Akiko  AYN                    2.0                    5.0   
465           DAN  AYM                    2.0                    5.0   
516        Edward  AYQ                    1.0                    5.0   
521  Shivam Patel  AYC                    1.0                    3.0   
523          Evan  AYG                    2.0                    3.0   

    What is your preferred way of saying one half? 7UP or Sierra Mist?  \
1                                            2-Jan                 7UP   
8                                            2-Jan                 7UP   
16                                           2-Jan         Sierra Mist   
17                                           2-Jan         Sierra Mist   
20                                             0.5                 7UP   
85                                           2-Jan                 7UP   
90                                           2-Jan                 7UP   
156                                          2-Jan         Sierra Mist   
179                                          2-Jan                 7UP   
247                                          2-Jan         Sierra Mist   
249                                          2-Jan                 7UP   
284                                            0.5                 7UP   
286                                          2-Jan                 7UP   
315                                          2-Jan                 7UP   
316                                          2-Jan                 7UP   
354                                          2-Jan                 7UP   
441                                          2-Jan         Sierra Mist   
465                                          2-Jan                 7UP   
516                                            0.5         Sierra Mist   
521                                            0.5                 7UP   
523                                          2-Jan                 7UP   

    Android or iOS? Apple or Orange? Beef or Pork? Your birthday is...  ...  \
1               iOS            Apple           NaN     On/After July 1  ...   
8           Android           Orange           NaN     On/After July 1  ...   
16              iOS            Apple          Beef     On/After July 1  ...   
17              iOS            Apple          Beef   On/Before June 30  ...   
20              iOS           Orange          Beef     On/After July 1  ...   
85              iOS            Apple          Beef     On/After July 1  ...   
90              iOS           Orange          Beef   On/Before June 30  ...   
156             iOS           Orange          Pork     On/After July 1  ...   
179             iOS           Orange          Beef   On/Before June 30  ...   
247      

### 🔬 Test Case Checkpoint 🔬

In [51]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert(len(df_goodsleep) == 503 ), "Double check your conditional used to create df_goodsleep from df_hello - remember, good sleep is response values of 6 hours or more"
assert(len(df_badsleep) == 21 ), "Double check your conditional used to create df_badsleep from df_hello - remember, bad sleep is response values of less than 6 hours"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Puzzle 3.2: Average Number of Classes by Group

Now find the **average number of classes** of each group (good sleep and bad sleep):

Hint: the `df['column name'].mean()` function returns the mean of all values in the specified column of `df`

In [54]:
classes = 'How many classes are you taking this semester?'
goodsleep_avg_classes = df_goodsleep["How many classes are you taking this semester?"].mean()
goodsleep_avg_classes

5.656

In [55]:
badsleep_avg_classes = df_badsleep["How many classes are you taking this semester?"].mean()
badsleep_avg_classes

5.285714285714286

### 🔬 Test Case Checkpoint 🔬

In [56]:
## == CHECKPOINT TEST CASES ==
# - This read-only cell contains test cases for your previous cell.
# - If this cell runs without any errors, you PASSED all test cases!
# - If this cell results in any errors, check your previous cell, make changes, and RE-RUN your code and then this cell.
assert(math.isclose(goodsleep_avg_classes,  5.656)), "The average number of classes for those with good sleep does not appear to have been correctly calculated"
assert(math.isclose(badsleep_avg_classes,  5.285714285714286)), "The average number of classes for those with bad sleep does not appear to have been correctly calculated"

## == SUCCESS MESSAGE ==
# You will only see this message (with the emoji showing) if you passed all test cases:
tada = "\N{PARTY POPPER}"
print(f"{tada} All tests passed! {tada}")

🎉 All tests passed! 🎉


### Analyis: Classes v. Sleeptime

**Q: What is the relationship between classes and average sleep time?  Can you think of a possible *confounding variable* in the observed relationship (or lack thereof) between classes and average sleep time?** Write at least three complete sentences.

*(✏️ I found that people with a good average sleep time are taking approximately 6 classes while people with bad average sleep time are taking 5. This shows that people with more classes tend to have a better average sleep time/sleep better. I couldn't really think of a confounding variable between classes and average sleep time. A possible causal link could be workload/level of the courses. The amount of classes and the difficulty of each class very likely leads to a good average sleep time. ✏️)*

<hr style="color: #DD3403;">

# Submission

You're almost done!  All you need to do is to commit your lab to GitHub:

1.  ⚠️ **Make certain to save your work.** ⚠️ To do this, go to **File => Save All**

2.  After you have saved, exit this notebook and follow the Canvas instructions to commit this lab to your Git repository!

3. Your TA will grade your submission and provide you feedback after the lab is due. :)